<a href="https://colab.research.google.com/github/gidee725/pyspark-project/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=3dacbaef320ceca28fd7acd4a7cec6343a037d8f62245c8c7ddd728898da4f41
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, row_number,  col
from pyspark.sql.window import Window

In [51]:
# Create a SparkSession
spark = SparkSession.builder.appName('All_week').getOrCreate()

In [52]:
# Define the path to the CSV file
csv_path = '/content/drive/MyDrive/data/all-weeks-countries.csv'

# Load the data
df = spark.read.format('csv').option('header','true').load(csv_path)

# Show the data
df.show()

+------------+------------+----------+--------+-----------+--------------------+--------------------+--------------------------+
|country_name|country_iso2|      week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------+------------+----------+--------+-----------+--------------------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21|   Films|          1|      Look Both Ways|                NULL|                         1|
|   Argentina|          AR|2022-08-21|   Films|          2|           Day Shift|                NULL|                         2|
|   Argentina|          AR|2022-08-21|   Films|          3|Bank Robbers: The...|                NULL|                         2|
|   Argentina|          AR|2022-08-21|   Films|          4|   The Next 365 Days|                NULL|                         1|
|   Argentina|          AR|2022-08-21|   Films|          5|The Angry Birds M...|                N

In [34]:
df.printSchema()

root
 |-- country_name: string (nullable = true)
 |-- country_iso2: string (nullable = true)
 |-- week: string (nullable = true)
 |-- category: string (nullable = true)
 |-- weekly_rank: string (nullable = true)
 |-- show_title: string (nullable = true)
 |-- season_title: string (nullable = true)
 |-- cumulative_weeks_in_top_10: string (nullable = true)



## Exercise 1
Data preparation - Load in te dataset, divide it into two dataframes for TV and
Film, modify the TV dataframe to replace any null values in the season_title column.

In [35]:
#Dividing the TV
df_tv = df.filter(df.category == 'TV')
df_tv.show()

+------------+------------+----------+--------+-----------+--------------------+--------------------+--------------------------+
|country_name|country_iso2|      week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------+------------+----------+--------+-----------+--------------------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21|      TV|          1| Pasión de Gavilanes|Pasión de Gavilan...|                         5|
|   Argentina|          AR|2022-08-21|      TV|          2|        Another Self|Another Self: Sea...|                         4|
|   Argentina|          AR|2022-08-21|      TV|          3| Pasión de Gavilanes|Pasión de Gavilan...|                        58|
|   Argentina|          AR|2022-08-21|      TV|          4|            Manifest|  Manifest: Season 1|                         6|
|   Argentina|          AR|2022-08-21|      TV|          5|         The Sandman|The Sandman: Seas

In [36]:
df_film = df.filter(df.category == 'Films')
df.show(10)

+------------+------------+----------+--------+-----------+--------------------+------------+--------------------------+
|country_name|country_iso2|      week|category|weekly_rank|          show_title|season_title|cumulative_weeks_in_top_10|
+------------+------------+----------+--------+-----------+--------------------+------------+--------------------------+
|   Argentina|          AR|2022-08-21|   Films|          1|      Look Both Ways|        NULL|                         1|
|   Argentina|          AR|2022-08-21|   Films|          2|           Day Shift|        NULL|                         2|
|   Argentina|          AR|2022-08-21|   Films|          3|Bank Robbers: The...|        NULL|                         2|
|   Argentina|          AR|2022-08-21|   Films|          4|   The Next 365 Days|        NULL|                         1|
|   Argentina|          AR|2022-08-21|   Films|          5|The Angry Birds M...|        NULL|                         1|
|   Argentina|          AR|2022-

In [38]:
df_tv = df_tv.na.fill({'season_title': 'Unknown'})
df_tv.filter(df_tv.season_title == 'Unknown').show()

+------------+------------+----------+--------+-----------+--------------------+------------+--------------------------+
|country_name|country_iso2|      week|category|weekly_rank|          show_title|season_title|cumulative_weeks_in_top_10|
+------------+------------+----------+--------+-----------+--------------------+------------+--------------------------+
|   Argentina|          AR|2022-07-10|      TV|          2|     Stranger Things|     Unknown|                         4|
|   Argentina|          AR|2022-06-12|      TV|         10|     Stranger Things|     Unknown|                         3|
|   Argentina|          AR|2022-06-05|      TV|          3|     Stranger Things|     Unknown|                         2|
|   Argentina|          AR|2022-05-29|      TV|          7|     Stranger Things|     Unknown|                         1|
|   Australia|          AU|2022-07-24|      TV|          8|     Stranger Things|     Unknown|                         9|
|   Australia|          AU|2022-

##Exercise 2
Making use of the "groupBy" and "where" methods, find the number of weeks the
show "Stranger Things" was in the Top 10 for the United Kingdom across all seasons.


In [40]:
# Cast 'weekly_rank' to integer
df_tv = df_tv.withColumn("weekly_rank", col("weekly_rank").cast("integer"))
df_tv.printSchema()

root
 |-- country_name: string (nullable = true)
 |-- country_iso2: string (nullable = true)
 |-- week: string (nullable = true)
 |-- category: string (nullable = true)
 |-- weekly_rank: integer (nullable = true)
 |-- show_title: string (nullable = true)
 |-- season_title: string (nullable = false)
 |-- cumulative_weeks_in_top_10: string (nullable = true)



In [42]:
# Filter the DataFrame for the show 'Stranger Things' in the 'show_title' column, 'United Kingdom' in the 'country_name' column, and weekly rank less than or equal to 10
df_stranger_things = df_tv.where((df_tv.show_title == 'Stranger Things') & (df_tv.country_name == 'United Kingdom') & (df_tv.weekly_rank <= 10))

# Group by 'season_title' and count the number of weeks for each season
df_stranger_things_seasons = df_stranger_things.groupBy('season_title').count()

# Show the result
df_stranger_things_seasons.show()

+-----------------+-----+
|     season_title|count|
+-----------------+-----+
|Stranger Things 4|   13|
|          Unknown|   10|
|Stranger Things 3|   12|
|Stranger Things 2|   11|
+-----------------+-----+



## Exercise 3
Produce a dataframe containing only the Top 25 TV seasons in the UK, based on
the number of weeks they spent in the Top 10.


In [47]:
# Filter the DataFrame for the shows with the weekly rank of top 10 in  the "weekly_rank" column  and from the 'United Kingdom' in the 'country_name' column,
df_top_25 = df_tv.where((df_tv.weekly_rank<= 10) & (df_tv.country_name == 'United Kingdom'))

# Group by 'season_title', count the number of weeks, and sort in descending order
df_top_25_seasons = df_top_25.groupBy('season_title').count().orderBy(desc('count'))

# Show the top 25 seasons
df_top_25_seasons.show(25)

+--------------------+-----+
|        season_title|count|
+--------------------+-----+
|             Unknown|   14|
|   Stranger Things 4|   13|
|     Ozark: Season 4|   12|
|   Stranger Things 3|   12|
|   Stranger Things 2|   11|
|Squid Game: Season 1|   10|
|Better Call Saul:...|   10|
|Maid: Limited Series|    9|
|Bridgerton: Season 2|    9|
|PAW Patrol: Season 6|    9|
|Inventing Anna: L...|    8|
| Money Heist: Part 5|    7|
|  Sex/Life: Season 1|    7|
|PIECES OF HER: Se...|    7|
|Sex Education: Se...|    6|
|Clickbait: Limite...|    6|
|Virgin River: Sea...|    6|
|Young Sheldon: : ...|    6|
|Young Sheldon: : ...|    6|
|The Lincoln Lawye...|    6|
|       You: Season 3|    6|
|PAW Patrol: Season 4|    6|
|The Umbrella Acad...|    5|
|Good Girls: Season 4|    5|
|The Witcher: Seas...|    5|
+--------------------+-----+
only showing top 25 rows



## Exercise 4
For the show "Young Sheldon", find the country where each season spent the
most time in the Top 10.


In [54]:
# Filter the DataFrame for the show 'Young Sheldon' and weeks in the top 10
df_young_sheldon = df_tv.where((df_tv.show_title == 'Young Sheldon') & (df_tv.weekly_rank <=10))

# Group by 'season_title' and 'country_name', and count the number of weeks
df_season_country = df_young_sheldon.groupBy('season_title', 'country_name').count()

# For each season, find the country with the maximum count
window = Window.partitionBy('season_title').orderBy(desc('count'))
df_top_country_per_season = df_season_country.withColumn('rank', row_number().over(window)).filter(col('rank') == 1).drop('rank')

# Show the result
df_top_country_per_season.show()

+--------------------+--------------+-----+
|        season_title|  country_name|count|
+--------------------+--------------+-----+
|Young Sheldon: : ...|        Canada|    6|
|Young Sheldon: : ...|United Kingdom|    6|
|Young Sheldon: : ...|United Kingdom|    4|
|Young Sheldon: Se...|   New Zealand|    1|
+--------------------+--------------+-----+



## Exercise 5
For each country, find the film that spent the most time in the Top 10.


In [60]:
# Filter the DataFrame for the  films that spent weeks in the top 10
df_films_top_10 = df_film.where(df_film.weekly_rank <= 10 )
df_country_film = df_films_top_10.groupBy('country_name', 'show_title').count()
# For each season, find the country with the maximum count
window = Window.partitionBy('country_name').orderBy(desc('count'))
df_top_countryfilm = df_country_film.withColumn('rank', row_number().over(window)).filter(col('rank') == 1).drop('rank')

# # Show the result
df_top_countryfilm.show()

+------------------+--------------------+-----+
|      country_name|          show_title|count|
+------------------+--------------------+-----+
|         Argentina|  Sonic the Hedgehog|    7|
|         Australia| Back to the Outback|    8|
|           Austria|Harry Potter and ...|    9|
|           Bahamas|  A Madea Homecoming|    9|
|           Bahrain|          Red Notice|    8|
|        Bangladesh|        Sooryavanshi|   21|
|           Belgium|    The Adam Project|    8|
|           Bolivia|               Shrek|   18|
|            Brazil|Hotel Transylvani...|    7|
|          Bulgaria|          Red Notice|   15|
|            Canada|How the Grinch St...|    7|
|             Chile|  Sonic the Hedgehog|    8|
|          Colombia|          Red Notice|   12|
|        Costa Rica|  Sonic the Hedgehog|   10|
|           Croatia|          Red Notice|   14|
|            Cyprus|          Red Notice|   12|
|    Czech Republic|The Secret Life o...|   14|
|           Denmark|          Red Notice

## Exercise 6
Calculate the number of weeks each film spent at the number 1 spot of each
country's Top 10 list. Then find the films that spent the most time in the number 1 spot for
each country.


In [65]:
# Filter the DataFrame for the  films that spent weeks in the number 1 spot
df_pos_1 = df_film.where((df_film.weekly_rank==1) & (df_film.weekly_rank<=10))
# for each country find the film with the most weeks in number 1
df_pos_1_film  = df_pos_1.groupBy('country_name', 'show_title').count()
window = Window.partitionBy('show_title').orderBy(desc('count'))
df_top_film_per_country = df_pos_1_film.withColumn('rank', row_number().over(window)).filter(col('rank')==1).drop('rank')
# Show the result
df_top_film_per_country.show()

+------------+--------------------+-----+
|country_name|          show_title|count|
+------------+--------------------+-----+
|  Bangladesh|                 '83|    2|
| Philippines|13 Hours: The Sec...|    2|
|     Ireland|          13 Minutes|    1|
|    Slovakia|14 Peaks: Nothing...|    1|
|     Nigeria|    2 Weeks in Lagos|    1|
|     Denmark|                  21|    1|
|       Egypt|           30. March|    1|
|      Greece|  365 Days: This Day|    2|
|    Thailand|             4 Kings|    2|
|      Norway|A Boy Called Chri...|    2|
|     Belgium|A Castle For Chri...|    1|
| Philippines|      A Faraway Land|    1|
|  Guadeloupe|  A Madea Homecoming|    2|
|     Nigeria|   A Naija Christmas|    2|
|   Indonesia|       A Perfect Fit|    1|
|     Romania|   A Perfect Pairing|    2|
|       Japan|A Quiet Place Par...|    1|
|     Bahamas|   A Score to Settle|    1|
|      Mexico|A Woman With No F...|    1|
|   Indonesia|     A World Without|    1|
+------------+--------------------